## Dataframe API

### Dataframe:
+ структурированная колоночная структура данных
+ может быть создана на основе:
  - локальной коллекции
  - файла (файлов)
  - базы данных
+ в Python работает значительно быстрее, чем RDD
+ под капотом использует RDD
+ позволяет выполнять произвольные SQL операции с данными
+ аналогично RDD являются ленивыми и неизменяеыми

### Из чего состоит Dataframe?
+ схема [pyspsark.sql.StructType](https://spark.apache.org/docs/2.4.7/api/python/pyspark.sql.html#pyspark.sql.types.StructType)
+ колонки [pyspark.sql.Column](https://spark.apache.org/docs/2.4.7/api/python/pyspark.sql.html#pyspark.sql.Column)
+ данные [pyspark.sql.Row](https://spark.apache.org/docs/2.4.7/api/python/pyspark.sql.html#pyspark.sql.Row)

## Основной управляющий объект в Spark SQL - [SparkSession](https://spark.apache.org/docs/2.4.7/api/python/pyspark.sql.html#pyspark.sql.SparkSession)

In [ ]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()

spark = SparkSession.builder.config(conf=conf).appName("Spark SQL").getOrCreate()

In [ ]:
spark

In [ ]:
spark.sparkContext

In [ ]:
%pylab inline

## Как создать DataFrame?

### Прочитать данные из файла

In [ ]:
spark.read

### Конвейер чтения данных
```python
spark.read\
     .format(...)\
     .option(key, value)\
     .option(key, value)\
     .load(path)
```

In [ ]:
%%time
df = spark.read\
          .format("csv")\
          .option("sep", "|")\
          .load("/data/spark/lecture06/ml-100k/u.user")

In [ ]:
%%time
rdd = spark.sparkContext.textFile("/data/spark/lecture06/ml-100k/u.user")

### Откуда такая разница?

In [ ]:
df

In [ ]:
df.show(1)

In [ ]:
df.take(5)

### Схема!

In [ ]:
from pyspark.sql.types import *

In [ ]:
schema = StructType(fields=[
    StructField("user_id", IntegerType()),
    StructField("age", IntegerType()),
    StructField("gender", StringType()),
    StructField("occupation", StringType()),
    StructField("zip", IntegerType())
])

In [ ]:
%%time
df = spark.read\
          .schema(schema)\
          .format("csv")\
          .option("sep", "|")\
          .load("/data/spark/lecture06/ml-100k/u.user")

In [ ]:
df

In [ ]:
df.printSchema()

In [ ]:
df.show(5)

### Схему также можно задавать в виде SQL DDL

In [ ]:
ddl_schema = """
    user_id INT,
    age INT,
    gender STRING,
    occupation STRING,
    zip INT
"""

In [ ]:
%%time
df = spark.read\
          .schema(ddl_schema)\
          .format("csv")\
          .option("sep", "|")\
          .load("/data/spark/lecture06/ml-100k/u.user")

In [ ]:
df.printSchema()

### Наличие схемы оборачивается еще одним приятным бонусом - правильно работает описательная статистика.

In [ ]:
df.summary().show()

### Что значит `nullable`?

In [ ]:
tiny_schema = StructType(fields=[
    StructField("id", IntegerType()),
    StructField("value", StringType(), nullable=False)
])

In [ ]:
df = spark.createDataFrame([[1, None], [3, "Hello"]], schema=tiny_schema)

In [ ]:
tiny_schema = StructType(fields=[
    StructField("id", IntegerType()),
    StructField("value", StringType(), nullable=True)
])

In [ ]:
df = spark.createDataFrame([[1, None], [3, "Hello"]], schema=tiny_schema)

In [ ]:
df.printSchema()

### На самом деле конвейер чтения выглядит так
```python
spark.read\
     .schema(schema)\
     .format(...)\
     .option(key, value)\
     .option(key, value)\
     .load(path)
```

### Для популярных источников есть удобные обертки

In [ ]:
df = spark.read.csv("/data/spark/lecture06/ml-100k/u.user", schema=schema, sep="|")

In [ ]:
df

In [ ]:
df.show(5)

### А самое классное, что через один API можно работать с множеством источников!
+ CSV
+ JSON
+ Hive
+ HBase
+ Cassandra
+ MySQL
+ PostgreSQL
+ Parquet
+ ORC
+ Kafka
+ ElasticSearch
+ Amazon S3
+ ...and more through custom connectors

### Create a DataFrame from an RDD, pandas.DataFrame or a list

In [ ]:
rdd = sc.textFile("/data/spark/lecture06/ml-100k/u.user").map(lambda x: x.split("|"))

In [ ]:
rdd.take(5)

In [ ]:
df = spark.createDataFrame(rdd)

### RDD нетипизирован и никакой схемы не имеет

In [ ]:
df

In [ ]:
df = spark.createDataFrame(rdd, schema=schema)

In [ ]:
df

In [ ]:
df.show(5)

### Type mismatch, maybe we should ignore schema verification??

In [ ]:
df = spark.createDataFrame(rdd, schema=schema, verifySchema=False)

In [ ]:
df.show(5)

### No chance, have to convert data to proper types

In [ ]:
rdd = rdd.map(lambda x: (int(x[0]), int(x[1]), x[2], x[3], int(x[4])))

In [ ]:
df = spark.createDataFrame(rdd, schema=schema)

In [ ]:
df.show(5)

## Работать будем со сгенерированным логом доступа

In [ ]:
!hdfs dfs -tail /data/spark/lecture06/logsM.txt

In [ ]:
log_schema = StructType(fields=[
    StructField("ip", StringType()),
    StructField("timestamp", LongType()),
    StructField("url", StringType()),
    StructField("size", IntegerType()),
    StructField("code", IntegerType()),
    StructField("ua", StringType())
])

In [ ]:
log = spark.read.csv("/data/spark/lecture06/logsM.txt", sep="\t", schema=log_schema).cache()

In [ ]:
log

In [ ]:
log.rdd.getNumPartitions()

In [ ]:
log = log.repartition(4)

In [ ]:
log.show(5, vertical=True, truncate=False)

## Проекции и фильтры
**Проекция** возвращает подмножество столбцов

**Фильтр** возвращает подмножество строк

In [ ]:
log.select(["ip", "timestamp", "url"])

In [ ]:
log.select("ip", "code").show(5)

In [ ]:
log.select(log.ip, log.code).show(5)

In [ ]:
log.ip

## Зачем нужны столбцы-объекты? Потому что у них есть методы!

In [ ]:
log.select(log.ip,
           log.code.alias("response")).show(5)

In [ ]:
import pyspark.sql.functions as f

In [ ]:
log.select("ip", 
           f.col("code").alias("response")).show(5)

## Pandas-like

In [ ]:
log[["ip", "code"]].show(5)

In [ ]:
log[[log.ip, log.code.alias("response")]].show(5)

## Фильтрация

In [ ]:
log.where("code = 200").show(5)

In [ ]:
log.filter(log.code == 200).show(5, truncate=False)

In [ ]:
log.filter("code == 200 AND url LIKE '%rambler%'").show(5, truncate=False, vertical=True)

In [ ]:
log.filter((log.code.isin([200, 404])) & (log.url.like("%rambler%"))).show(5)

## Pandas

In [ ]:
log[(log.code == 200) & (log.url.like("%rambler%"))].show(5)

## И все вместе

In [ ]:
log[(log.code == 200) & (log.url.like("%rambler%"))][["ip", "code"]].show(5)

## А SQL-то можно писать?

In [ ]:
query = """
SELECT ip, code FROM log_table 
WHERE code == 200 AND url LIKE '%rambler%'
"""

In [ ]:
spark.sql(query).show(5)

### При запуске на YARN `SparkSession` автоматически запускается с поддержкой HIVE. Поэтому надо зарегистрировать `DataFrame` как таблицу во внутреннем каталоге Spark SQL

In [ ]:
log.createOrReplaceTempView("log_table")

In [ ]:
spark.catalog.listTables()

In [ ]:
spark.sql(query).show(5)

### Какая разница между Temp view и Global Temp View?
`SparkSession` - не singleton в отличие от `SparkContext`

In [ ]:
spark2 = spark.newSession()

In [ ]:
spark2 is spark

In [ ]:
spark2.sparkContext is spark.sparkContext

## Функции
Не все вычисления можно реализовать стандартным SQL. Здесь на помощь приходят функции (встроенные или пользовательские). Встроенные функции находятся в модуле [`pyspark.sql.functions`](https://spark.apache.org/docs/2.4.7/api/python/pyspark.sql.html#module-pyspark.sql.functions) 

In [ ]:
log.select("ua", f.length("ua")).show(5)

### Функции возвращают объект типа `Column`

In [ ]:
f.length("ua")

In [ ]:
log.select("ua", f.length("ua").alias("length")).show(5)

### Довольно часто возникает ошибка с неверными именами столбцов

In [ ]:
log.select(f.concat("url", "?utm_medium=email")).show(5)

### Нужно убедиться в правильности имени или типа

In [ ]:
log.select(f.concat("url", f.lit("?utm_medium=email")).alias("newurl")).show(5, False)

## Взрывы!
Посчитаем word count

In [ ]:
log.select("ua", f.split("ua", " ").alias("word_list")).show(5, False, True)

### Прелесть ленивых вычислений и строгой типизации

In [ ]:
log.select("ua", f.split("ua", " ").alias("word_list")).printSchema()

### К элементам сложных типов можно получить доступ

In [ ]:
log.select("ua", f.split("ua", " ").alias("word_list"))\
   .select(f.col("word_list")[0].alias("first_word"), f.col("word_list")[1].alias("second_word"))\
   .show(5)

In [ ]:
log.select("ua", f.split("ua", " ").alias("word_list"))\
   .select(f.explode("word_list").alias("word"))\
   .groupby("word").count()\
   .orderBy("count", ascending=False)\
   .show(5)

### Самое время посмотреть в Spark UI!

## Joins

In [ ]:
!hdfs dfs -tail /data/spark/lecture06/ipDataM.txt

In [ ]:
ip_schema = StructType(fields=[
    StructField("ip", StringType()),
    StructField("region", StringType())
])

In [ ]:
ips = spark.read.csv("/data/spark/lecture06/ipDataM.txt", schema=ip_schema, sep="\t").cache()

In [ ]:
ips.show(5)

### Трюк для отключения автоматической оптимизации

In [ ]:
spark.sql("SET spark.sql.autoBroadcastJoinThreshold = 100500")

In [ ]:
log_with_regions = log.join(ips, on="ip", how="inner")

In [ ]:
log_with_regions

In [ ]:
log_with_regions.show(5)

### Можно делать не только equi-join, но и по произвольному выражению

In [ ]:
ips_with_ts = ips.withColumn("timestamp", f.lit(20140127041332).cast("bigint"))

In [ ]:
log.join(ips_with_ts, on=((log.ip == ips_with_ts.ip) & (log.timestamp > ips_with_ts.timestamp))).count()

In [ ]:
log_with_regions.count()

### А что там с партициями?

In [ ]:
(log.rdd.getNumPartitions(),
 ips.rdd.getNumPartitions(),
 log_with_regions.rdd.getNumPartitions())

In [ ]:
spark.conf.get("spark.sql.shuffle.partitions")

In [ ]:
log_with_regions = log_with_regions.coalesce(4).cache()

### По-умолчанию Spark SQL использует алгоритм SortMergeJoin

In [ ]:
log_with_regions.explain(extended=True)

### Если одна из таблиц мала, то можно реализовать map-side join через broadcast

In [ ]:
log_with_regions = log.join(f.broadcast(ips), on="ip", how="inner")

In [ ]:
log_with_regions.explain()

## Аггрегация
```python
df.groupBy(*cols)\
  .agg(*expressions)
```

In [ ]:
log_with_regions.groupBy("region")\
                .agg(f.count("ip").alias("count"))\
                .orderBy("count", ascending=False)\
                .show(10)

In [ ]:
log_with_regions.groupBy("region")\
                .count()\
                .withColumnRenamed("count", "row_count")\
                .orderBy("row_count", ascending=False)\
                .show(10)

In [ ]:
length_stat = log_with_regions.groupBy(f.length("url").alias("url_length"))\
                              .agg(f.count("*").alias("row_count"))\
                              .orderBy("row_count", ascending=False)\
                              .toPandas()

In [ ]:
length_stat

## UDF (User Defined Functions)
The function type of the UDF can be one of the following:
+ **SCALAR**. A scalar UDF defines a transformation: One or more `pandas.Series` -> A `pandas.Series`. calar UDFs are used with `pyspark.sql.DataFrame.withColumn()` and `pyspark.sql.DataFrame.select()`
+ **GROUPED_MAP**. A grouped map UDF defines transformation: A `pandas.DataFrame` -> A `pandas.DataFrame`. Grouped map UDFs are used with `pyspark.sql.GroupedData.apply()`

In [ ]:
@f.pandas_udf(StringType())
def encode_http_status(codes):
    mapping = {
        1: "info",
        2: "success",
        3: "redirect",
        4: "client error",
        5: "server error"
    }
    return (codes // 100).replace(mapping)

In [ ]:
log.withColumn("http_status", encode_http_status("code"))\
   .groupBy("http_status").count().show()

In [ ]:
spark.stop()